<a href="https://colab.research.google.com/github/lubolacko/ML-v-Pythone-priklady/blob/main/ML_v_pr%C3%ADkladoch_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML v príkladoch III

Tenzory sú základným stavebným kameňom strojového učenia. Ich úlohou je reprezentovať údaje formou čísel. Sú to viacrozmerné vektory a sú flexibilné v tom, čo môže reprezentovať. Napríklad obrázok s rozlíšením 512 x 384 pixelov reprezentujeme ako tenzor tvaru [3, 384, 512], kde jednotlivé rozmery vektora reprezentujú atribúty [farba_RGB, výška, šírka], nakoľko na obrázku sú 3 farebné kanály RGB(červený, zelený, modrý). Obrázok sa transformuje na tenzorovú reprezentáciu tak, že pre každý bod sa určia tri farebné kanály a jeho súradnice v osi Y a X.

Vektor je jednorozmerný tenzor, ktorý môže obsahovať viac čísel. Pre úplnosť máme aj tenzor nulového rozmeru, čiže skalár, alebo ľudovo povedané jedno číslo. Počet rozmerov, ktoré má tenzor v PyTorch, na prvý pohľad zistíte podľa počtu otváracích hranatých zátvoriek "["

In [1]:
import torch
vektor = torch.tensor([1, 3, 5, 7, 9, 11])
print(vektor)
print("rozmer tenzora", vektor.ndim)


tensor([ 1,  3,  5,  7,  9, 11])
rozmer tenzora 1


Ukážeme aj príklad vytvorenia trojrozmerného tenzora. Pripomíname, že počet rozmerov, ktoré má tenzor v PyTorch, zistíte podľa počtu otváracích hranatých zátvoriek, takže musia byť tri. Napríklad:

In [2]:
tenzor = torch.tensor([[[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]]])
print(tenzor)
print("rozmer tenzora", tenzor.ndim)
tenzor.shape


tensor([[[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]])
rozmer tenzora 3


torch.Size([1, 3, 3])

# Konverzia textu na tenzor
Ak chceme trénovať, alebo dotrénovať neurónovú sieť na bloku textu, je potrebné text prekonvertovať na číselné hodnoty. V reálnych príkladoch by sme text načítali z textového, PDF, HTML, či iného súboru. My pre jednoduchosť použijeme textový reťazec


In [5]:

text = "Ondrej Tráva gazda v Kameňanoch \
zavesil gazdovanie na klinec \
a oddal sa na špekuláciu \
Gazdovstvo viedol jeho syn \
a Ondrej sa zaoberal kupectvom"
print(text)

Ondrej Tráva gazda v Kameňanoch zavesil gazdovanie na klinec a oddal sa na špekuláciu Gazdovstvo viedol jeho syn a Ondrej sa zaoberal kupectvom


Následne je potrebné text vyčistiť, to znamená vynechať  nealfanumerické znaky a zoskupenia znakov. Na tento účel sa používa knižnica Natural Language Toolkit (ntlk), konkrétne funkciu word_tokenize(). Toto budeme riešiť v niektorej ďalšej časti, teraz sa zameriame na konverziu slov na čísla, preto budeme konverziu robiť na už vyčistenom texte. Po eliminovaní duplicitných slov získame toľko hodnôt, koľko je v texte unikátnych slov. V našom texte máme minimálne štyri duplicitné slová: Ondrej, sa, na, a. Funkcia set() poskytne zoznam unikátnych slov, čiže vytvoríme slovník.

In [10]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
slova = word_tokenize(text.lower())
slovnik = set(slova)
print (slovnik)


{'gazdovanie', 'kameňanoch', 'gazdovstvo', 'ondrej', 'kupectvom', 'zavesil', 'v', 'zaoberal', 'jeho', 'tráva', 'viedol', 'oddal', 'klinec', 'na', 'špekuláciu', 'gazda', 'sa', 'syn', 'a'}


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Teraz potrebujeme slová, čiže textové reťazce na číselné hodnoty, pretože neurónové siete nepracujú s prirodzeným jazykom, ale s „jazykom matematiky“. To je zároveň aj odpoveď na prípadnú otázku, či záleží na tom v akom jazyku sú texty na trénovanie. Nezáleží, jedinou podmienkou je aby s jazyky nemiešali, napríklad slovenčina s češtinou. Pri trénovaní neurónovej siete nás zaujíma len to ako zvyknú slová, prípadne znaky po sebe nasledovať.  Pomocou funkcie enumerate() slová z našej dátovej množiny očíslujeme.

In [19]:
ix_slov = {slovo: i for i, slovo in enumerate(slovnik)}
ix_slov


{'gazdovanie': 0,
 'kameňanoch': 1,
 'gazdovstvo': 2,
 'ondrej': 3,
 'kupectvom': 4,
 'zavesil': 5,
 'v': 6,
 'zaoberal': 7,
 'jeho': 8,
 'tráva': 9,
 'viedol': 10,
 'oddal': 11,
 'klinec': 12,
 'na': 13,
 'špekuláciu': 14,
 'gazda': 15,
 'sa': 16,
 'syn': 17,
 'a': 18}

Teraz náš text reprezentovaný postupnosťou slov zmeníme na postupnosť čísel.

In [20]:
zoznam_ix_slov = [ix_slov[slovo] for slovo in slova]
zoznam_ix_slov[:10]


[3, 9, 15, 6, 1, 5, 0, 13, 12, 18]

Kde 3 = Ondrej, 9 = tráva, 15 = gazda....  Index konkrétneho slova môžeme zistiť príkazom

In [21]:
ix_slov['gazda']

15

Následne prekonvertujeme pole indexov slov, ktoré reprezentuje náš text na tenzor, ktorý môžeme ďalej použiť na trénovanie neurónovej siete

In [22]:
tenzor_slova = torch.tensor([zoznam_ix_slov], dtype=torch.long)
tenzor_slova

tensor([[ 3,  9, 15,  6,  1,  5,  0, 13, 12, 18, 11, 16, 13, 14,  2, 10,  8, 17,
         18,  3, 16,  7,  4]])

Pre text knihy Rivers of Babylon od Petra Pišťanka, ktorú sme  použili vo viacerých príkladoch trénovania neurónovej siete na našom webe je 90 374 slov, pričom slovník je tvorený 19 679 unikátnymi slovami.

Môžeme použiť inú stratégiu a text na trénovanie neurónovej siete konvertovať na číselné hodnoty nie po slovách, ale po jednotlivých písmenách. Aj v tomto prípade vynecháme nealfanumerické znaky Pre náš text o gazdovi Ondrejovi:


In [25]:
znaky = sorted(list(set(text)))
unikatnych_znakov = len(znaky)
print(''.join(znaky))
print(unikatnych_znakov)

 GKOTabcdeghijklmnoprstuvyzáňš
30


In [ ]:
# mapovanie znakov na indexy
znaky_na_ix = { ch:i for i,ch in enumerate(znaky) }
znaky_na_ix

na zakódovanie textu na čísla a použijeme funkciu typu lambda.

In [32]:
text_na_cisla = lambda t: [znaky_na_ix[c] for c in t]
print(text_na_cisla(text))

[3, 17, 8, 20, 9, 13, 0, 4, 20, 27, 24, 5, 0, 10, 5, 26, 8, 5, 0, 24, 0, 2, 5, 16, 9, 28, 5, 17, 18, 7, 11, 0, 26, 5, 24, 9, 21, 12, 15, 0, 10, 5, 26, 8, 18, 24, 5, 17, 12, 9, 0, 17, 5, 0, 14, 15, 12, 17, 9, 7, 0, 5, 0, 18, 8, 8, 5, 15, 0, 21, 5, 0, 17, 5, 0, 29, 19, 9, 14, 23, 15, 27, 7, 12, 23, 0, 1, 5, 26, 8, 18, 24, 21, 22, 24, 18, 0, 24, 12, 9, 8, 18, 15, 0, 13, 9, 11, 18, 0, 21, 25, 17, 0, 5, 0, 3, 17, 8, 20, 9, 13, 0, 21, 5, 0, 26, 5, 18, 6, 9, 20, 5, 15, 0, 14, 23, 19, 9, 7, 22, 24, 18, 16]


Text pretransformovaný na čísla prekonvertujeme na tenzor knižnice torch.

In [33]:
tenzory_textu = torch.tensor(text_na_cisla(text), dtype=torch.long)
print(tenzory_textu.shape)
tenzory_textu

torch.Size([143])


tensor([ 3, 17,  8, 20,  9, 13,  0,  4, 20, 27, 24,  5,  0, 10,  5, 26,  8,  5,
         0, 24,  0,  2,  5, 16,  9, 28,  5, 17, 18,  7, 11,  0, 26,  5, 24,  9,
        21, 12, 15,  0, 10,  5, 26,  8, 18, 24,  5, 17, 12,  9,  0, 17,  5,  0,
        14, 15, 12, 17,  9,  7,  0,  5,  0, 18,  8,  8,  5, 15,  0, 21,  5,  0,
        17,  5,  0, 29, 19,  9, 14, 23, 15, 27,  7, 12, 23,  0,  1,  5, 26,  8,
        18, 24, 21, 22, 24, 18,  0, 24, 12,  9,  8, 18, 15,  0, 13,  9, 11, 18,
         0, 21, 25, 17,  0,  5,  0,  3, 17,  8, 20,  9, 13,  0, 21,  5,  0, 26,
         5, 18,  6,  9, 20,  5, 15,  0, 14, 23, 19,  9,  7, 22, 24, 18, 16])

Vstupné údaje, v našom prípade text v podobe tenzora, samozrejme reálny, nie takýto krátky ako máme v príklade rozdelíme na trénovaciu a validačnú množinu a môžeme začať trénovať neurónovú sieť, prípadne dotrénovať už existujúcu neurónovú sieť. V príklade 90%  textu od začiatku budú trénovacie dáta. Pre kontrolu zistíme veľkosť každej množiny

In [34]:
# 90%  textu od začiatku budú trénovacie dáta
n = int(0.9*len(tenzory_textu))
tren_data = tenzory_textu[:n]
test_data = tenzory_textu[n:]
print("Trénovacia množina:",tren_data.shape)
print("Testovacia množina:",test_data.shape)

Trénovacia množina: torch.Size([128])
Testovacia množina: torch.Size([15])


V budúcom pokračovaní ukážeme ako naprogramovať model neurónovej siete pomocou funkcií knižnice PyTorch